In [185]:
import capstone
import keystone
import shutil
import struct
import re

In [186]:
def make_idx_dict(names):
    d = {} 
    for i, n in enumerate(names):
        d[n] = i

    return d

def two_way_dict(dictionary):
    for k in list(dictionary.keys()):
        dictionary[dictionary[k]] = k

    return dictionary

def dict_safe_get(dictionary, key):
    try:
        return dictionary[key]
    except KeyError:
        if isinstance(key, str):
            return 0
        else:
            return 'UNKNOWN'
        
def expand_rt_dict(dictionary):
    for k in list(dictionary.keys()):
        if k[0] == 'x':
            rx_pattern = re.compile(r'r(\d)+', 0)
            val = dictionary[k]

            if rx_pattern.search(val):
                val += 'd'
            else:
                val = 'e' + val[1:]

            dictionary['w' + k[1:]] = val

    dictionary['xzr'] = 0
    dictionary['wzr'] = 0

    reg_p = '('
    for k in dictionary.keys():
        reg_p += k + '|'
    reg_p = reg_p[:-1] + ')'

    # for k,v in dictionary.items():
        # print(k,v)

    return reg_p, dictionary


In [187]:
class Const:
    HEADER_SIZE = 64 

In [188]:
class ElfHeader:
    unpacked_data = None

    fields = [
        "e_ident",
        "e_type",
        "e_machine",
        "e_version",
        "e_entry",
        "e_phoff",
        "e_shoff",
        "e_flags",
        "e_ehsize",
        "e_phentsize",
        "e_phnum",
        "e_shentsize",
        "e_shnum",
        "e_shstrndx",
    ]

    idx_dict = make_idx_dict(fields)

    format = (
        # e_ident (16 bytes), e_type (2 bytes), e_machine (2 bytes), e_version (4 bytes)
        '< 16s H H I' +
        # e_entry (8 bytes), e_phoff (8 bytes), e_shoff (8 bytes), e_flags (4 bytes)
        'Q Q Q I' +
        # e_ehsize (2 bytes), e_phentsize (2 bytes), e_phnum (2 bytes), e_shentsize (2 bytes)
        'H H H H' +
        # e_shnum (2 bytes), e_shstrndx (2 bytes)
        'H H'
    )

    def print():
        for i, f in enumerate(ElfHeader.fields):
            print(f'{f}: {ElfHeader.unpacked_data[i]}')
    
    def read_elf_header():
        if ElfFile.data[:4] != b'\x7fELF':
            raise ValueError("Not a valid ELF file.")
        
        ElfHeader.unpacked_data = list(struct.unpack(ElfHeader.format, ElfFile.data[:Const.HEADER_SIZE]))

    def overwrite_elf_header(file_path):
        amd_machine = 0x003e
        ElfHeader.unpacked_data[2] = amd_machine

        packed_data = struct.pack(ElfHeader.format, *ElfHeader.unpacked_data)

        with open(file_path, 'wb') as f:
            f.write(packed_data)
            f.write(ElfFile.data[Const.HEADER_SIZE:])

    def get(name):
        idx = ElfHeader.idx_dict[name]
        return ElfHeader.unpacked_data[idx]

In [189]:
class SectionHeader:
    delete_pattern = \
        re.compile(r'(.note.gnu.property|.eh.frame)', 0)

    sh_types = two_way_dict({
        'SHT_NULL' : 0, 
        'SHT_PROGBITS' : 1, 
        'SHT_SYMTAB' : 2, 
        'SHT_STRTAB' : 3, 
        'SHT_RELA' : 4, 
        'SHT_HASH' : 5, 
        'SHT_DYNAMIC' : 6, 
        'SHT_NOTE' : 7, 
        'SHT_NOBITS' : 8, 
    })

    fields = [
        "sh_name",      # Section name (index into section header string table)
        "sh_type",      # Section type
        "sh_flags",     # Section attributes
        "sh_addr",      # Virtual address in memory
        "sh_offset",    # Offset in file
        "sh_size",      # Size of section
        "sh_link",      # Link to other section
        "sh_info",      # Miscellaneous information
        "sh_addralign", # Address alignment boundary
        "sh_entsize"    # Size of entries, if section has table
    ]

    format = (
        # sh_name (4 bytes), sh_type (4 bytes), sh_flags (8 bytes), sh_addr (8 bytes)
        '< I I Q Q' +  
        # sh_offset (8 bytes), sh_size (8 bytes), sh_link (4 bytes), sh_info (4 bytes)
        'Q Q I I' +    
        # sh_addralign (8 bytes), sh_entsize (8 bytes)
        'Q Q' 
    )

    idx_dict = make_idx_dict(fields)

    shstroff = None
    
    def __init__(self, offset, verbose = False):
        self.unpacked_data = list(
            struct.unpack(
                SectionHeader.format, 
                ElfFile.data[offset : offset + ElfHeader.get('e_shentsize')],
            )
        )

        sh_off = self.get('sh_offset')
        self.section_data = ElfFile.data[sh_off : sh_off + self.get('sh_size')]
        self.type = SectionHeader.sh_types[self.get('sh_type')]

        self.name = None

        if verbose:
            self.print()

    def print(self):
        for i, f in enumerate(SectionHeader.fields):
            print(f'{f}: {self.unpacked_data[i]}')
        print(self.type)

    def get(self, name):
        idx = SectionHeader.idx_dict[name]
        return self.unpacked_data[idx]
    
    def set_name(self, verbose = False):
        self.name = ElfFile.find_string(self.get('sh_name'), True)

        if verbose:
            print(self.name)

In [190]:
class Sym:
    size = 0x18

    format = (
        # st_name(4 bytes), st_info(1 byte), st_other(1 byte)
        '< I B B' +
        # st_shndx(2 bytes), st_value(8 bytes), st_size(8 bytes)
        'H Q Q'
    )

    fields = [
        'st_name',
        'st_info',
        'st_other',
        'st_shndx',
        'st_value',
        'st_size',
    ]

    stbsym = two_way_dict({
        'STB_LOCAL' : 0,
        'STB_GLOBAL' : 1,
        'STB_WEAK' : 2,
    })

    sttsym = two_way_dict({
        'STT_NOTYPE' : 0,
        'STT_OBJECT' : 1,
        'STT_FUNC' : 2,
        'STT_SECTION' : 3,
        'STT_FILE' : 4,
    })
        
    idx_dict = make_idx_dict(fields)

    def __init__(self, offset, verbose = False):
        self.unpacked_data = struct.unpack(Sym.format, ElfFile.data[offset : offset + Sym.size])
        self.offset = offset

        self.name = ElfFile.find_string(self.get('st_name'), sh_string = False)
        self.binding, self.type = Sym.get_binding_and_type(self.get('st_info'))

        if(verbose):
            self.print()

    def get_binding_and_type(st_info):
        binding = (st_info >> 4) & 0xF  
        symbol_type = st_info & 0xF
        
        return dict_safe_get(Sym.stbsym, binding), dict_safe_get(Sym.sttsym, symbol_type)

    def print(self):
        for i, f in enumerate(Sym.fields):
            print(f'{f}: {self.unpacked_data[i]}')

        print(self.name, self.binding, self.type)

    def overwrite_sym(self, offset):
        pass

    def get(self, name):
        idx = Sym.idx_dict[name]
        return self.unpacked_data[idx]

    def collect_sym_entries(sh):
        base_offset = offset = sh.get('sh_offset')
        size = sh.get('sh_size')
        entsize = sh.get('sh_entsize')
        sym_entries = []

        while offset < base_offset + size:
            sym_entries += [Sym(offset)]
            offset += entsize

        return sym_entries

In [ ]:
class Rela:
    code_types = two_way_dict({
        'R_AARCH64_CALL26' : 283,
        'R_AARCH64_ADD_ABS_LO12_NC' : 277,	
        'R_AARCH64_ADR_PREL_PG_HI21' : 275,
        'R_AARCH64_ABS64' : 257,
        # not overlapping, so i keep it in the same dict
        'R_AMD64_64' : 1,
        'R_AMD64_PC32' : 2,
        'R_AMD64_32' : 10,
    })

    size = 0x18

    format = '<QQq'  # r_offset, r_info, r_addend

    fields = ['r_offset', 'r_info', 'r_addend']

    idx_dict = make_idx_dict(fields)

    #define ELF64_R_SYM(i)((i) >> 32)
    def R_SYM(i):
        return i >> 32
    
    #define ELF64_R_TYPE(i)((i) & 0xf f f f f f f f L)
    def R_TYPE(i):
        return i & 0xffffffff

    #define ELF64_R_INFO(s, t)(((s) << 32) + ((t) & 0xf f f f f f f f L))
    def R_INFO(s, t):
        return (s << 32) + (t & 0xffffffff)

    def __init__(self, offset, verbose = False):
        self.unpacked_data = list(
            struct.unpack(Rela.format, ElfFile.data[offset : offset + Rela.size])
        )
        self.offset = offset
        info = self.get('r_info')
        self.sym = Rela.R_SYM(info)
        self.type = Rela.code_types[Rela.R_TYPE(info)]

        if verbose:
            self.print()

    def print(self):
        for i, f in enumerate(Rela.fields):
            print(f'{f}: {self.unpacked_data[i]}')

        print(f'{self.sym=}')
        print(f'{self.type=}')

    def overwrite_rela(
            self, 
            offset = None, 
            symbol = None, 
            type = None, 
            addend = None,
        ):
        if offset:
            self.unpacked_data[0] = offset
        if symbol:
            self.sym = symbol
        if type:
            self.type = type 
        if addend:
            self.unpacked_data[2] = addend

        self.unpacked_data[1] = Rela.R_INFO(self.sym, Rela.code_types[type])
        
    def get(self, name):
        idx = Rela.idx_dict[name]
        return self.unpacked_data[idx]

    def collect_rela_entries(sh):
        base_offset = offset = sh.get('sh_offset')
        size = sh.get('sh_size')
        entsize = sh.get('sh_entsize')
        rela_entries = {}

        while offset < base_offset + size:
            rela = Rela(offset)
            rela_entries[rela.get('r_offset')] = rela
            offset += entsize

        return rela_entries


In [192]:
class Comparator:
    # for capstone output
    hex_pattern = '(0x[0-9a-f]+)' 

    prolog = """
        stp x29, x30, [sp, #-prologue_shift]!
        mov x29, sp
    """

    epilog = """
        ldp x29, x30, [sp], #prologue_shift
        ret
    """

    escape_chars = '[]'

    def unify(text, pattern = False):
        if pattern:
            for char in Comparator.escape_chars:
                text = text.replace(f'{char}', fr'\{char}')

        text = text.replace('prologue_shift', Comparator.hex_pattern)

        tokens = text.strip().split()

        unified = ''
        for t in tokens:
            unified += t + ' '

        return unified 
    
    def compare_part(code, is_prolog = True, verbose = False):
        unified_code = Comparator.unify(code)

        pattern = getattr(Comparator, 'prolog' if is_prolog else 'epilog')
        unified_pattern = Comparator.unify(pattern, True)

        if verbose: 
            print(unified_code)
            print(unified_pattern)

        compiled_pattern = re.compile(unified_pattern, re.IGNORECASE)
        return compiled_pattern.search(unified_code)

    # def count_lines(chars, code, from_end = False):
    #     lengths = []
    #     for line in code.splitlines():
    #         length = 0
    #         for word in line.split():
    #             length += len(word)
    #         lengths += [length]

    #     lengths = reversed(lengths) if from_end else lengths

    #     lines = 0
    #     for l in lengths:
    #         if chars == 0:
    #             return lines

    #         chars -= l
    #         lines += 1

    #     assert False, 'Function detection failed'

    def check_function(code):
        match_p = Comparator.compare_part(code, True)
        match_e = Comparator.compare_part(code, False)

        if match_p and match_e:
            span = (match_p.span()[0], match_e.span()[1])

            assert match_p.group(1) == match_p.group(1), "Prologue shift doesn't match"

            if span == (0, len(code) - 1):
                return match_p.group(1) 

            assert False, (span(), len(code) - 1, 'Assignment conditions not fullfilled.')
            
        return None

In [193]:
class Translator:
    prolog_x86 = """
    push rbp
    mov rbp, rsp
    sub rsp, #prologue_shift
    """

    epilog_x86 = """
    mov rax, rdi
    leave
    ret
    """

    def count_functions(code_section):
        code = Translator.disassemble_code(code_section, show_offsets = False)
        return Comparator.check_function(code)

    def disassemble_code(code_section, show_offsets = True, verbose = False):
        # AArch64 architecture
        md = capstone.Cs(capstone.CS_ARCH_ARM64, capstone.CS_MODE_ARM)
        instructions = md.disasm(code_section, 0)

        code = ""
        for insn in instructions:
            off = f"0x{insn.address:x}:\t" if show_offsets else ""
            code_line = f"{off}{insn.mnemonic}\t{insn.op_str}"

            code += code_line + "\n"

            if verbose:
                print(code_line)
        
        return code
    
    def translate_code(code_section, p_shift, rela_section = None):
        code = Translator.disassemble_code(code_section, True).splitlines()
        code_x86 = Translator.prolog_x86.replace('#prologue_shift', p_shift)

        md = capstone.Cs(capstone.CS_ARCH_ARM64, capstone.CS_MODE_ARM)
        md.detail = True
        instructions = md.disasm(code_section, 0)

        fields = [
        'address', 'bytes', 'errno', 'group', 'group_name', 'groups', 'id',
        'insn_name', 'mnemonic', 'op_count', 'op_find', 'op_str', 'reg_name',
        'reg_read', 'reg_write', 'regs_access', 'regs_read', 'regs_write', 'size'
        ]

        inst_list = []
        for insn in instructions:
            inst_list += [insn]

        for insn in inst_list[2:-2]:
            off = f"0x{insn.address:x}:\t" 

            code_line = f"{off}{insn.mnemonic}\t{insn.op_str}"
            code_line_x86 = ParseInsn.parse(insn, rela_section)

            code += code_line + "\n"
            code_x86 += code_line_x86 

            print(code_line)
            print(code_line_x86)

            # print(ParseInsn.parse(insn, rela_section))

        return code_x86 + Translator.epilog_x86

    def assemble_code(code):
        # separate assembly instructions by ; or \n
        CODE = b"INC ecx; DEC edx"
        
        try:
            ks = keystone.Ks(keystone.KS_ARCH_X86, keystone.KS_MODE_64)
            encoding, count = ks.asm(CODE)
            print("%s = %s (number of statements: %u)" %(CODE, encoding, count))

        except keystone.KsError as e:
            print("ERROR: %s" %e)

In [194]:
class ParseInsn:
    cond_mapping = {
        'eq' : 'e',
        'ne' : 'ne',
        'hs' : 'ae',
        'lo' : 'b',
        'mi' : 's',
        'pl' : 'ns',
        'vs' : 'o',
        'vc' : 'no',
        'hi' : 'a',
        'ls' : 'be',
        'ge' : 'ge',
        'lt' : 'l',
        'gt' : 'g',
        'le' : 'le',
    }

    reg_p, register_translation = expand_rt_dict({
        'x0' : 'rdi',
        'x1' : 'rsi',
        'x2' : 'rdx',
        'x3' : 'rcx',
        'x4' : 'r8',
        'x5' : 'r9',
        'x9' : 'rax',
        'x10' : 'r10',
        'x29' : 'rbp',
        'x19' : 'rbx',
        'x20' : 'r12',
        'x21' : 'r13',
        'x22' : 'r14',
        'x23' : 'r15',
        'sp' : 'rsp',
    })

    str_p = r'([#a-z0-9]+)'
    hex_p = r'#([xa-f0-9]+)'

    @staticmethod
    def isreg64(reg):
        return reg[0] == 'x' or reg[0] == 's'
        
    @staticmethod
    def get_sizeq(reg):
        if ParseInsn.isreg64(reg):
            return 'qword_ptr'
        else:
            return 'dword_ptr'

    @staticmethod
    def is_reg(str):
        return re.compile(ParseInsn.reg_p).search(str)

    @staticmethod
    def parse(insn, rela = None):
        if insn.mnemonic[:2] == 'b.':
            return ParseInsn.bcond(insn, insn.mnemonic[2:], rela)
        else:
            handle_fun = getattr(ParseInsn, insn.mnemonic)
            return handle_fun(insn, rela)

    @staticmethod
    def ldr(insn : capstone.CsInsn, rela : dict[int, Rela] = None):
        ptn = f'{ParseInsn.reg_p}, ' 
        ptn += rf'\[{ParseInsn.reg_p}, {ParseInsn.hex_p}\]'
        ptn = re.compile(ptn)

        reg1, reg2, op2d = ptn.search(insn.op_str).groups()

        op1 = ParseInsn.register_translation[reg1]
        op2b = ParseInsn.register_translation[reg2]
        sq = ParseInsn.get_sizeq(reg1)

        return f'mov {op1}, {sq} [{op2b} + {op2d}]\n'

    @staticmethod
    def str(insn : capstone.CsInsn, rela : dict[int, Rela] = None):
        ptn = f'{ParseInsn.reg_p}, ' 
        ptn += rf'\[{ParseInsn.reg_p}, {ParseInsn.hex_p}\]'
        ptn = re.compile(ptn)

        reg1, reg2, op2d = ptn.search(insn.op_str).groups()

        op1 = ParseInsn.register_translation[reg1]
        op2b = ParseInsn.register_translation[reg2]
        sq = ParseInsn.get_sizeq(reg1)

        return f'mov {sq} [{op2b} + {op2d}], {op1}\n'

    @staticmethod
    def adrp(insn : capstone.CsInsn, rela : dict[int, Rela] = None):
        ptn = f'{ParseInsn.reg_p}, ' 
        ptn += rf'{ParseInsn.hex_p}'
        ptn = re.compile(ptn)

        reg1, _ = ptn.search(insn.op_str).groups()

        # rela[insn.address].print()
        rela[insn.address].overwrite_rela(type = 'R_AMD64_32')

        op1 = ParseInsn.register_translation[reg1]

        # the displacement forces the assembler to use a 32-bit immediate; it is relocated
        ret = f'lea {op1}, [rip + 0x7fffffff]\n' 

        # set 12 lowest bits to 0
        ret += f'and {op1}, ~0xfff\n' 

        return ret

    @staticmethod
    def mov_or_cmp(insn : capstone.CsInsn, op_name, rela = None):
        ptn = f'{ParseInsn.reg_p}, ' 
        ptn = re.compile(ptn + rf'{ParseInsn.str_p}')

        op1, op2 = ptn.search(insn.op_str).groups()
        op1 = ParseInsn.register_translation[op1]

        if ParseInsn.is_reg(op2):
            op2 = ParseInsn.register_translation[op2]
        else:
            op2 = op2[1:]

        return f'{op_name} {op1}, {op2}\n'

    @staticmethod
    def mov(insn, rela = None):
        return ParseInsn.mov_or_cmp(insn, 'mov', rela)

    @staticmethod
    def cmp(insn, rela = None):
        return ParseInsn.mov_or_cmp(insn, 'cmp', rela)

    @staticmethod
    def add(insn : capstone.CsInsn, rela : dict[int, Rela] = None):
        ptn = f'{ParseInsn.reg_p}, ' 
        ptn += f'{ParseInsn.reg_p}, ' 
        ptn = re.compile(ptn + rf'{ParseInsn.str_p}')

        op1_old, op2, op3 = ptn.search(insn.op_str).groups()
        op1 = ParseInsn.register_translation[op1_old]
        op2 = ParseInsn.register_translation[op2]

        has_rela = insn.address in rela.keys() 
        if ParseInsn.is_reg(op3):
            op3 = ParseInsn.register_translation[op3]
            if has_rela:
                rela[insn.address].overwrite_rela(type = 'R_AMD64_32')
        else:
            op3 = op3[1:]

        if op1 == op2:
            return f'add {op1}, {op3}\n'
        elif op1 == op3:
            return f'add {op1}, {op2}\n'
        elif has_rela:
            tmp = 'r11' if ParseInsn.isreg64(op1_old) else 'r11d'
            ret = f'mov {tmp}, 0x7fffffff\n' # the immediate is relocated
            ret += f'and {tmp}, 0xfff\n'
            return ret + f'add {op1}, {tmp}\n'
        else:
            ret = f'mov {op1}, {op2}\n'       
            return ret + f'add {op1}, {op3}\n'       

    @staticmethod
    def bl(insn : capstone.CsInsn, rela : dict[int, Rela] = None):
        rela[insn.address].overwrite_rela(type = 'R_AMD64_PC32')

        # the offset is relocated
        ret = 'call 7fffffff\n' 
        # put the return value in the register to which x0 maps'
        ret += 'mov rdi, rax\n'
        return ret
    
    @staticmethod
    def b(insn : capstone.CsInsn, rela : dict[int, Rela] = None):
        ptn = re.compile(f'{ParseInsn.hex_p}')

        imm = ptn.search(insn.op_str).groups()

        return f'jmp {imm}\n'
    
    @staticmethod
    def bcond(insn : capstone.CsInsn, cond, rela : dict[int, Rela] = None):
        ptn = re.compile(f'{ParseInsn.hex_p}')

        imm = ptn.search(insn.op_str).groups()
        cond = ParseInsn.cond_mapping[cond]

        return f'j{cond} {imm}\n'

In [ ]:
class ElfFile:
    data = None

    section_headers = []
    symbols = []
    rela_dict = {}

    shstroff = None
    sh_dict = {}

    def setup(file_path):
        with open(file_path, 'rb') as f:
            ElfFile.data = f.read()

    def read_elf_header():
        ElfHeader.read_elf_header()

    def find_string(relative_offset, sh_string = False):
        str_offset = relative_offset

        if sh_string:
            str_offset += ElfFile.shstroff 
        else:
            str_offset += ElfFile.sh_dict['.strtab'].get('sh_offset')

        str_end = ElfFile.data.find(b'\x00', str_offset)
        str_len = str_end - str_offset

        str = struct.unpack(f'{str_len}s', ElfFile.data[str_offset : str_end])[0]

        return str
        
    def read_section_headers(verbose = False):
        for i in range(ElfHeader.get('e_shnum')):
            offset = ElfHeader.get('e_shoff') + i * ElfHeader.get('e_shentsize')

            ElfFile.section_headers += [SectionHeader(offset)]
        
        shstrns = ElfFile.section_headers[ElfHeader.get('e_shstrndx')]
        ElfFile.shstroff = shstrns.get('sh_offset')

        null_sh = ElfFile.section_headers[0]

        for i in range(len(ElfFile.section_headers)):
            sh = ElfFile.section_headers[i]
            sh.set_name()

            decoded_name = sh.name.decode('utf-8')
            ElfFile.sh_dict[decoded_name] = sh

            if SectionHeader.delete_pattern.search(decoded_name):
                ElfFile.section_headers[i] = null_sh

            if verbose:
                ElfFile.section_headers[i].print()

    def read_symbols():
        sym_sh = ElfFile.sh_dict['.symtab']
        ElfFile.symbols = Sym.collect_sym_entries(sym_sh)

    def read_rela():
        for sh in ElfFile.section_headers:
            if (sh.type == 'SHT_RELA'):
                rela_entries = Rela.collect_rela_entries(sh)
                ElfFile.rela_dict[sh.name] = rela_entries
            
    def look_for_section(name):
        for sh in ElfFile.section_headers:
            if sh.name == name:
                return sh

    def find_code_sections():
        for s in ElfFile.symbols:
            if s.type == 'STT_FUNC': 
                sh = ElfFile.section_headers[s.get('st_shndx')]

                offset = s.get('st_value')
                code = sh.section_data[offset : offset + s.get('st_size')]

                rela_name = b'.rela' + sh.name
                exists = rela_name in ElfFile.rela_dict.keys()
                rela = ElfFile.rela_dict[rela_name] if exists else None
                
                if p_shift := Translator.count_functions(code):
                    code_x86 = Translator.translate_code(code, p_shift, rela)

In [196]:
input = 'test-aarch64.o'  
output = 'out.o'
good_output = 'test-aarch64-x64.o'  

shutil.copy(input, output)

ElfFile.setup(input)
ElfFile.read_elf_header()
ElfFile.read_section_headers()
ElfFile.read_symbols()
ElfFile.read_rela()
ElfFile.find_code_sections()

# print(Sym.sttsym.items())

0x8:	str	w0, [sp, #0x1c]
mov dword_ptr [rsp + 0x1c], edi

0xc:	str	w1, [sp, #0x18]
mov dword_ptr [rsp + 0x18], esi

0x10:	ldr	w1, [sp, #0x1c]
mov esi, dword_ptr [rsp + 0x1c]

0x14:	ldr	w0, [sp, #0x18]
mov edi, dword_ptr [rsp + 0x18]

0x18:	add	w0, w1, w0
add edi, esi

0x1c:	str	w0, [sp, #0x2c]
mov dword_ptr [rsp + 0x2c], edi

0x20:	ldr	w0, [sp, #0x1c]
mov edi, dword_ptr [rsp + 0x1c]

55834574848
2
55834574850
0x24:	bl	#0x24
call 7fffffff
mov rdi, rax

21474836480
10
21474836490
0x28:	adrp	x0, #0
lea rdi, [rip + 0x7fffffff]
and rdi, ~0xfff

0x2c:	add	x0, x0, #0
add rdi, 0

60129542144
2
60129542146
0x30:	bl	#0x30
call 7fffffff
mov rdi, rax

0x34:	ldr	w0, [sp, #0x18]
mov edi, dword_ptr [rsp + 0x18]

55834574848
2
55834574850
0x38:	bl	#0x38
call 7fffffff
mov rdi, rax

21474836480
10
21474836490
0x3c:	adrp	x0, #0
lea rdi, [rip + 0x7fffffff]
and rdi, ~0xfff

0x40:	add	x0, x0, #0
add rdi, 0

60129542144
2
60129542146
0x44:	bl	#0x44
call 7fffffff
mov rdi, rax

0x48:	ldr	w0, [sp, #0x2c]
mov ed